In order to easily extract references from articles, I first needed to determine which articles are available as full texts in the PMC Open Access subset. To do this, I downloaded the list of all articles in the Open Access subset and determined which PMIDs from the MTI ML Dataset are also in the Open Access subset.

Next, I downloaded the full texts for all PMIDs in the 2013 MTI ML Dataset that are also in PMC's Open Access subset (determined above).

References were extracted from full text XML files using the Beautiful Soup package. Initially, I tried to use xmltodict to do this, which ended up being very difficult. However, I was exposed to all the different formats of the full text XMLs and was able to see that references are always under the "back" tag, and they always have the "pub-id" identifier in their tags. 